### ------------------------------------------------------------------------------------------------------------

In [ ]:
import os

import pandas as pd

if 'COLAB_GPU' in os.environ:
    !git clone https://github.com/impulsecorp/PickStocks.git
    !mv PickStocks/*.py .
    !mv PickStocks/data .
    !pip install -U -qq -r PickStocks/requirements.txt

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import system
from system import *
# small hack to prevent Colab error
try:
    from datablock import *
except:
    from datablock import *
seed

In [ ]:
%pylab inline

### Setup

In [ ]:
data_timeperiod = 'D'
data = get_data('SPY', period=data_timeperiod, nrows=None)
data = procdata_lite(data)

In [ ]:
# for inspectiion
print(data.shape)
data.head()

In [ ]:
system.train_set_end = 0.75 # percentage point specifying the training set end point (1.0 means all data is training set)
system.val_set_end = 1.0   # percentage point specifying the validation set end point (1.0 means no test set)
system.balance_data = 1
system.scale_data = 1

### ------------------------------------------------------------------------------------------------------------

### Base classifier test

In [ ]:
# Train LogisticRegression classifier on train data
clf, scaler = train_classifier(LogisticRegression, data)

In [ ]:
# Test on val data
equity, pf, trades = qbacktest(clf, scaler, data)

In [ ]:
trades[0:10]

### ------------------------------------------------------------------------------------------------------------

### Neural Network Training

In [ ]:
X_train, y_train = get_clean_Xy(data.iloc[0:int(data.shape[0] * system.train_set_end)])
clf = None

In [ ]:
clf, scaler = train_classifier(NeuralNetClassifierWrapper, data, input_dim=X_train.shape[1],
                               dropout_prob=0.25, num_layers=3, learning_rate=0.001, 
                               hidden_dim=16, n_epochs=100)

In [ ]:
# Test on val data
equity, pf, trades = qbacktest(clf, scaler, data)

In [ ]:
trades.head()

### ------------------------------------------------------------------------------------------------------------